In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: [Number]
### Team Member Names: Arya Patel, [Name] & [Name]
### Team Strategy Chosen: RISKY

In [39]:
# Replace 'Tickers_Example.csv' with the actual ticker CSV file
ticker_file = 'Tickers_Example.csv'

# Read the CSV file into a pandas DataFrame with a single column
tickers_df = pd.read_csv(ticker_file, header=None, names=['Tickers'])

# Set the single column as the index
tickers_df.set_index('Tickers', inplace=True)

# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}
hist_column = {}
long_hist_column = {}
ticker_obj_lst = []

long_start_date = "2018-10-31"
start_date = "2023-01-01"
end_date = "2023-10-31"

tickers_df = tickers_df[~tickers_df.index.duplicated(keep='first')]
# Add info to dict
for ticker in tickers_df.index:
    yf_ticker = yf.Ticker(ticker)
    info_column[ticker] = yf_ticker.fast_info
    hist_column[ticker] = yf_ticker.history(start=start_date, end=end_date)
    long_hist_column[ticker] = yf_ticker.history(start=long_start_date, end=end_date)
    print('.', end='') # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df['History'] = pd.Series(hist_column)
tickers_df['Long History'] = pd.Series(long_hist_column)
tickers_df

...

AGN: No timezone found, symbol may be delisted
AGN: No timezone found, symbol may be delisted


............

CELG: No timezone found, symbol may be delisted
CELG: No timezone found, symbol may be delisted


......

MON: No timezone found, symbol may be delisted
MON: No timezone found, symbol may be delisted


........

RTN: No timezone found, symbol may be delisted
RTN: No timezone found, symbol may be delisted


............

,Info,History,Long History
Tickers,,,
AAPL,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
ABBV,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
ABT,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open Hig...
ACN,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
AGN,"lazy-loading dict with keys = ['currency', 'da...","Empty DataFrame Columns: [Open, High, Low, Clo...","Empty DataFrame Columns: [Open, High, Low, Clo..."
AIG,"lazy-loading dict with keys = ['currency', 'da...",Open Hig...,Open Hig...
AMZN,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
AXP,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
BA,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...


In [41]:
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
threshold_day = 18
days_in_month = []

CAD_stocks = []
USD_stocks = []

for ticker in tickers_df.index:
    if tickers_df['History'][ticker].empty:
        print(ticker, "is delisted.")
    else:
        print(ticker)
        group_months = tickers_df["History"][ticker].reset_index().groupby(pd.Grouper(key="Date", freq='M'))
        dfs = [group for _, group in group_months]
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))

        if tickers_df['Info'][ticker]['currency'] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif tickers_df['History'][ticker]['Volume'].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", tickers_df.iloc['Info'][ticker]['lastVolume'], "and average days of", np.mean(days_in_month))
        else:
            if tickers_df['Info'][ticker]['currency'] == 'USD':
                USD_stocks.append(ticker)
            elif tickers_df['Info'][ticker]['currency'] == 'CAD':
                CAD_stocks.append(ticker)
                                
print("\nValid CAD Stocks:\n",CAD_stocks)
print("\nValid USD Stocks:\n",USD_stocks)

AAPL
ABBV
ABT
ACN
AGN is delisted.
AIG
AMZN
AXP
BA
BAC
BIIB
BK
BLK
BMY
C
CAT
CELG is delisted.
CL
KO
LLY
LMT
MO
MON is delisted.
MRK
PEP
PFE
PG
PM
PYPL
QCOM
RTN is delisted.
RY.TO
SHOP.TO
T.TO
TD.TO
TXN
UNH
UNP
UPS
USB
YCBD

Valid CAD Stocks:
 ['RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO']

Valid USD Stocks:
 ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'YCBD']


In [124]:
def create_df(df):
    """
    This function iterates through the filtered list of stocks and creates a new dict with weekly ticker closing prices
    :param df: filtered stocks DataFrame
    :return: dict containing ticker closing prices resampled to weekly data
    """
    dic = {}
    for i in range(len(df)):
        tick = df[i]
        t_hist = tickers_df.loc[tickers_df['Ticker'] == tick, 'History'].values[0]
        dic[tick] = t_hist.Close
    return dic

CAD_prices = pd.DataFrame(create_df(CAD_stocks))
USD_prices = pd.DataFrame(create_df(USD_stocks))
print(CAD_prices)
print(USD_prices)

                                RY.TO    SHOP.TO       T.TO      TD.TO
Date                                                                  
2023-01-03 00:00:00-05:00  122.739281  48.790001  25.214359  84.689217
2023-01-04 00:00:00-05:00  123.832161  50.610001  25.520916  85.790459
2023-01-05 00:00:00-05:00  123.467865  48.830002  25.492176  84.384216
2023-01-06 00:00:00-05:00  125.040092  49.560001  25.884954  84.345154
2023-01-09 00:00:00-05:00  125.970009  49.810001  25.827473  84.081490
...                               ...        ...        ...        ...
2023-10-24 00:00:00-04:00  109.029999  71.870003  22.360001  76.830002
2023-10-25 00:00:00-04:00  109.110001  66.900002  22.280001  76.919998
2023-10-26 00:00:00-04:00  110.209999  64.529999  22.320000  77.410004
2023-10-27 00:00:00-04:00  108.470001  64.349998  22.010000  76.160004
2023-10-30 00:00:00-04:00  110.120003  64.860001  22.360001  77.190002

[208 rows x 4 columns]
                                 AAPL        ABBV    

In [43]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=long_start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_df["Long History"][ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in (CAD_stocks + USD_stocks):
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


C:\Users\matth\AppData\Local\Temp\ipykernel_19368\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_19368\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_19368\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample(

[('YCBD', 2.1401685163092337),
 ('SHOP.TO', 1.8105683661552798),
 ('C', 1.556663285550515),
 ('BA', 1.4762120231389992),
 ('BAC', 1.385100395377466),
 ('PYPL', 1.379281690066955),
 ('AAPL', 1.303634337150048),
 ('BLK', 1.2835912420806057),
 ('ACN', 1.230450335902667),
 ('QCOM', 1.216744471283755),
 ('AXP', 1.2146623598896331),
 ('AMZN', 1.1719942897819302),
 ('UPS', 1.1049476854133506),
 ('UNP', 1.1025178052139566),
 ('BK', 1.0953386362677164),
 ('CAT', 1.0879434823389227),
 ('AIG', 1.0351014884958258),
 ('USB', 1.0267306495616106),
 ('TXN', 1.0168284492333297),
 ('PM', 0.7766934948964196),
 ('ABT', 0.6933945879264342),
 ('MO', 0.6535227630328517),
 ('TD.TO', 0.6386755008347169),
 ('UNH', 0.6321136429750597),
 ('PFE', 0.630532043975632),
 ('LMT', 0.5987251858942175),
 ('KO', 0.5908109646775296),
 ('PEP', 0.5726423222067502),
 ('RY.TO', 0.5664457791088118),
 ('ABBV', 0.5070216207597805),
 ('PG', 0.47479134336190965),
 ('CL', 0.4606666136182472),
 ('T.TO', 0.45966954501727925),
 ('BMY', 

In [126]:
corr_df = USD_prices.corr()
corr_df

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,PFE,PG,PM,PYPL,QCOM,TXN,UNH,UNP,UPS,USB
AAPL,1.000000,-0.455763,-0.063601,0.784656,-0.034845,0.860951,0.141936,0.233318,-0.537339,-0.038402,...,-0.757230,0.559687,-0.275050,-0.563829,-0.205598,-0.014606,0.174806,0.400463,-0.172988,-0.625834
ABBV,-0.455763,1.000000,-0.149337,-0.361314,-0.095435,-0.366960,-0.325377,-0.071822,0.062586,-0.320042,...,0.186988,0.125832,0.186051,0.173923,-0.035314,-0.141626,0.293657,0.028789,0.119344,0.198391
ABT,-0.063601,-0.149337,1.000000,-0.048712,0.085519,-0.147335,0.241272,0.511352,0.527833,0.574184,...,0.490066,0.278376,0.596683,0.542668,0.283691,0.481726,-0.278426,0.188468,0.501109,0.311556
ACN,0.784656,-0.361314,-0.048712,1.000000,0.430474,0.915055,0.086466,0.320034,-0.248349,-0.226160,...,-0.606601,0.491362,-0.115419,-0.563529,-0.121161,-0.074584,0.297466,0.656415,-0.345826,-0.277916
AIG,-0.034845,-0.095435,0.085519,0.430474,1.000000,0.308702,0.024047,0.083182,0.446401,-0.378056,...,-0.084891,-0.021282,0.348650,-0.147586,0.083182,-0.275556,0.354767,0.599287,-0.395261,0.503794
AMZN,0.860951,-0.366960,-0.147335,0.915055,0.308702,1.000000,-0.013925,0.231038,-0.436795,-0.284251,...,-0.786084,0.575480,-0.255279,-0.710772,-0.296963,-0.230388,0.303137,0.620372,-0.443868,-0.452214
AXP,0.141936,-0.325377,0.241272,0.086466,0.024047,-0.013925,1.000000,0.519013,0.525260,0.160244,...,0.240700,-0.331807,0.401433,0.474498,0.698624,0.681538,-0.422761,0.079263,0.618747,0.422699
BA,0.233318,-0.071822,0.511352,0.320034,0.083182,0.231038,0.519013,1.000000,0.409708,0.146759,...,0.218868,0.277583,0.381783,0.262684,0.390110,0.595254,-0.270005,0.569326,0.536535,0.313482
BAC,-0.537339,0.062586,0.527833,-0.248349,0.446401,-0.436795,0.525260,0.409708,1.000000,0.118415,...,0.704170,-0.417819,0.698568,0.722790,0.670502,0.461503,-0.246965,0.174176,0.485520,0.939639
BIIB,-0.038402,-0.320042,0.574184,-0.226160,-0.378056,-0.284251,0.160244,0.146759,0.118415,1.000000,...,0.425941,0.020821,0.053735,0.333885,0.069916,0.349658,-0.325847,-0.403371,0.366823,-0.137713


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.